In [44]:
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

import numpy as np
import matplotlib.pyplot as plt
import os

In [72]:
import tensorflow as tf

In [45]:
from sklearn.preprocessing import LabelEncoder

In [46]:
data = []
labels = []


files = list(os.listdir('E:/Jupyter/Face mask detection/dataset/training/with_mask'))
for i in files:
    img = load_img('E:/Jupyter/Face mask detection/dataset/training/with_mask/' + i, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    data.append(img)
    labels.append('with_mask')

    
files = list(os.listdir('E:/Jupyter/Face mask detection/dataset/training/without_mask'))
for i in files:
    img = load_img('E:/Jupyter/Face mask detection/dataset/training/without_mask/' + i, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    data.append(img)
    labels.append('without_mask')

In [47]:
len(data)

1103

In [48]:
'''data_test = []
labels_test = []


files = list(os.listdir('E:/Jupyter/Face mask detection/dataset/test/with_mask'))
for i in files:
    img = load_img('E:/Jupyter/Face mask detection/dataset/test/with_mask/' + i, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    data_test.append(img)
    labels_test.append('with_mask')

    
files = list(os.listdir('E:/Jupyter/Face mask detection/dataset/test/without_mask'))
for i in files:
    img = load_img('E:/Jupyter/Face mask detection/dataset/test/without_mask/' + i, target_size=(224, 224))
    img = img_to_array(img)
    img = preprocess_input(img)
    data_test.append(img)
    labels_test.append('without_mask')'''

"data_test = []\nlabels_test = []\n\n\nfiles = list(os.listdir('E:/Jupyter/Face mask detection/dataset/test/with_mask'))\nfor i in files:\n    img = load_img('E:/Jupyter/Face mask detection/dataset/test/with_mask/' + i, target_size=(224, 224))\n    img = img_to_array(img)\n    img = preprocess_input(img)\n    data_test.append(img)\n    labels_test.append('with_mask')\n\n    \nfiles = list(os.listdir('E:/Jupyter/Face mask detection/dataset/test/without_mask'))\nfor i in files:\n    img = load_img('E:/Jupyter/Face mask detection/dataset/test/without_mask/' + i, target_size=(224, 224))\n    img = img_to_array(img)\n    img = preprocess_input(img)\n    data_test.append(img)\n    labels_test.append('without_mask')"

In [49]:
data = np.array(data, dtype="float32")
labels = np.array(labels)

In [50]:
'''data_test = np.array(data_test, dtype='float32')
labels = np.array(labels)'''

"data_test = np.array(data_test, dtype='float32')\nlabels = np.array(labels)"

In [51]:
data.shape

(1103, 224, 224, 3)

In [52]:
labels.shape

(1103,)

## Model Training

In [53]:
model = Sequential()

In [54]:
model.add(MobileNetV2(weights="imagenet", include_top=False,
	input_shape=(224, 224, 3)))

In [55]:
model.add(AveragePooling2D(pool_size=(7,7)))

In [56]:
model.add(Flatten())

In [57]:
model.add(Dense(128, activation='relu'))

In [58]:
model.add(Dropout(0.25))

In [59]:
model.add(Dense(2, activation='softmax'))

In [60]:
model.layers

In [61]:
model.layers[0].trainable = False

In [62]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 7, 7, 1280)        2257984   
_________________________________________________________________
average_pooling2d_2 (Average (None, 1, 1, 1280)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               163968    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 258       
Total params: 2,422,210
Trainable params: 164,226
Non-trainable params: 2,257,984
______________________________________

In [63]:
lb = LabelEncoder()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

#labels_test = lb.fit_transform(labels_test)
#labels_test = to_categorical(labels_test)

In [64]:
data_gen = ImageDataGenerator(rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [65]:
epochs = 20
batch_size = 32

steps_per_epoch_training = len(data)//batch_size
#validation_step = len(data_test)//batch_size

In [66]:
opt = Adam(lr=0.005)
model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=['accuracy'])

In [67]:
data.shape

(1103, 224, 224, 3)

In [78]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.9):
            print("\nReached 60% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()

In [ ]:
H = model.fit(
        data_gen.flow(data, labels, batch_size=batch_size),
        steps_per_epoch=steps_per_epoch_training,
        epochs=30,  callbacks=[callbacks])

Epoch 1/30
34/34 [==============================] - 38s 1s/step - loss: 0.2658 - accuracy: 0.8777
Epoch 2/30
34/34 [==============================] - 39s 1s/step - loss: 0.2597 - accuracy: 0.8777
Epoch 3/30
34/34 [==============================] - 39s 1s/step - loss: 0.2853 - accuracy: 0.8665
Epoch 4/30
34/34 [==============================] - 39s 1s/step - loss: 0.2635 - accuracy: 0.8778
Epoch 5/30
34/34 [==============================] - 39s 1s/step - loss: 0.2462 - accuracy: 0.8880
Epoch 6/30
34/34 [==============================] - 39s 1s/step - loss: 0.2504 - accuracy: 0.8786
Epoch 7/30
34/34 [==============================] - 39s 1s/step - loss: 0.2722 - accuracy: 0.8824
Epoch 8/30
34/34 [==============================] - 38s 1s/step - loss: 0.2504 - accuracy: 0.8908
Epoch 9/30
26/34 [=====================>........] - ETA: 7s - loss: 0.2306 - accuracy: 0.8945

In [ ]:
model.save('Mobilenet_mask_classifier_all_30_epochs.h5')

In [ ]:
"""model_json = model.to_json()

with open("model.json", "w") as json_file:
    json_file.write(model_json)"""